In [1]:
!cd /Users/lc/Developer/kuzu &&  make clean-python-api && make release NUM_THREADS=20 

rm -rf tools/python_api/build
cd external && \
	mkdir -p build && \
	cd build && \
	cmake    -DCMAKE_BUILD_TYPE=Release ../ && \
	cmake --build . --config Release -- -j 20
-- Configuring done
-- Generating done
-- Build files have been written to: /Users/lc/Developer/kuzu/external/build
[100%] Built target apache_arrow
mkdir -p build/release && \
	cd build/release && \
	cmake    -DCMAKE_BUILD_TYPE=Release ../.. && \
	cmake --build . --config Release -- -j 20
-- pybind11 v2.11.0 dev1
-- Configuring done
-- Generating done
-- Build files have been written to: /Users/lc/Developer/kuzu/build/release
[  0%] Built target kuzu_transaction
[  0%] Built target utf8proc
[  0%] Built target kuzu_catalog
[  0%] Built target kuzu_common_csv_reader
[  1%] Built target kuzu_common_task_system
[  1%] Built target kuzu_common_data_chunk
[  2%] Built target kuzu_common_arrow
[  2%] Built target kuzu_binder
[  3%] Built target kuzu_binder_expression
[  4%] Built target kuzu_common_vector
[  5%] Built tar

In [2]:
import sys
sys.path.append('/Users/lc/Developer/kuzu/tools/python_api/build')
import kuzu

In [3]:
import torch_geometric.transforms as T
from torch_geometric.datasets import OGB_MAG


dataset = OGB_MAG(root='/Users/lc/Developer/ogb_pytorch', preprocess='metapath2vec', transform=T.ToUndirected())
data = dataset[0]

In [4]:
import torch

In [5]:
DB_PATH = '/Users/lc/Developer/kuzu_ogb_mag'
db = kuzu.Database(DB_PATH)

In [6]:
conn = kuzu.Connection(db)

In [7]:
fs, gs = db.get_torch_geometric_remote_backend()

In [8]:
data

HeteroData(
  paper={
    x=[736389, 128],
    year=[736389],
    y=[736389],
    train_mask=[736389],
    val_mask=[736389],
    test_mask=[736389]
  },
  author={ x=[1134649, 128] },
  institution={ x=[8740, 128] },
  field_of_study={ x=[59965, 128] },
  (author, affiliated_with, institution)={ edge_index=[2, 1043998] },
  (author, writes, paper)={ edge_index=[2, 7145660] },
  (paper, cites, paper)={ edge_index=[2, 10792672] },
  (paper, has_topic, field_of_study)={ edge_index=[2, 7505078] },
  (institution, rev_affiliated_with, author)={ edge_index=[2, 1043998] },
  (paper, rev_writes, author)={ edge_index=[2, 7145660] },
  (field_of_study, rev_has_topic, paper)={ edge_index=[2, 7505078] }
)

In [9]:
from torch_geometric.loader import NeighborLoader


In [10]:
loader_kuzu = NeighborLoader(
    data=(fs, gs),
    num_neighbors={key.edge_type : [30] * 2 for key in gs.get_all_edge_attrs()},
    # Use a batch size of 128 for sampling training nodes of type paper
    batch_size=128,
    input_nodes=('paper', torch.arange(0, data['paper'].num_nodes)),
    input_time=None
)

In [12]:
import time
times = []
for i in range(10):
    start = time.time()
    next(iter(loader_kuzu))
    end = time.time()
    if i > 0:
        times.append(end - start)
        print("=====")
        print("Total time", end - start)
print("=====")
print(sum(times) / len(times))

Index cast time:  0.00010013580322265625
Scan time:  0.0022096633911132812
Reshape time:  0.0
Tensor cast time:  1.5735626220703125e-05
Index cast time:  0.0004417896270751953
Scan time:  0.046154022216796875
Reshape time:  2.09808349609375e-05
Tensor cast time:  1.4066696166992188e-05
Index cast time:  7.200241088867188e-05
Scan time:  0.001847982406616211
Reshape time:  0.0
Tensor cast time:  6.9141387939453125e-06
Index cast time:  5.1975250244140625e-05
Scan time:  0.0020720958709716797
Reshape time:  1.1920928955078125e-06
Tensor cast time:  1.1920928955078125e-05
Index cast time:  2.288818359375e-05
Scan time:  2.9802322387695312e-05
Reshape time:  0.0
Tensor cast time:  1.9073486328125e-06
Index cast time:  5.0067901611328125e-06
Scan time:  0.00015020370483398438
Reshape time:  1.0013580322265625e-05
Tensor cast time:  2.1457672119140625e-06
Index cast time:  1.0013580322265625e-05
Scan time:  0.00044608116149902344
Reshape time:  0.0
Tensor cast time:  1.1920928955078125e-06
I

In [12]:
next(iter(loader_kuzu))

HeteroData(
  paper={
    num_nodes=35999,
    id=[35999],
    x=[35999, 128],
    year=[35999],
    y=[35999],
    input_id=[128],
    batch_size=128
  },
  institution={
    num_nodes=333,
    id=[333],
    x=[333, 128]
  },
  author={
    num_nodes=5709,
    id=[5709],
    x=[5709, 128]
  },
  field_of_study={
    num_nodes=2792,
    id=[2792],
    x=[2792, 128]
  },
  (paper, has_topic, field_of_study)={ edge_index=[2, 20795] },
  (author, affiliated_with, institution)={ edge_index=[2, 0] },
  (institution, rev_affiliated_with, author)={ edge_index=[2, 942] },
  (field_of_study, rev_has_topic, paper)={ edge_index=[2, 12417] },
  (paper, cites, paper)={ edge_index=[2, 22022] },
  (author, writes, paper)={ edge_index=[2, 7942] },
  (paper, rev_writes, author)={ edge_index=[2, 9075] }
)

In [13]:
loader_pyg = NeighborLoader(
    data=data,
    num_neighbors={key.edge_type : [30] * 2 for key in gs.get_all_edge_attrs()},
    # Use a batch size of 128 for sampling training nodes of type paper
    batch_size=128,
    input_nodes=('paper', torch.arange(0, data['paper'].num_nodes)),
    input_time=None
)

In [14]:
import time
times = []
for i in range(10):
    start = time.time()
    next(iter(loader_pyg))
    end = time.time()
    if i > 0:
        times.append(end - start)
        print(end - start)
print("=====")
print(sum(times) / len(times))

0.046340227127075195
0.04477691650390625
0.04415106773376465
0.04438495635986328
0.043502092361450195
0.04590892791748047
0.043756961822509766
0.04406404495239258
0.0437161922454834
=====
0.04451126522488064
